In [88]:
# 필요한 모듈 불러오기
%matplotlib inline

import time
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
from sklearn.metrics import confusion_matrix
import sklearn.preprocessing as preprocessing

sns.set()
sns.set_style('white') 
sns.set_color_codes(palette='pastel')

#plt.rcParams['axes.unicode_minus'] = False
#path = 'C:/Windows/Fonts/malgun.ttf'
#font_name = mpl.font_manager.FontProperties(fname=path, size=50).get_name()
#print(font_name)
#plt.rc('font', family=font_name)

In [89]:
#데이토 로드 
root_dir = '../[classification]loan_use/'
data = pd.read_csv(os.path.join(root_dir,'loan_data.csv'))

In [90]:
data.head()

,AGE,TOT_LOAN,TOT_LOAN_CRD,LOAN_BNK,LOAN_CPT,CRDT_CNT,GUARN_CNT,INCOME,LOAN_CRD_CNT,LATE_RATE,...,CALL_TIME,TEL_COST_MON,MOBILE_PRICE,SUSP_DAY,LATE_TEL,COMB_COMM,SEX,PAY_METHOD,JOB,TARGET
0,40,6,6,6,0,0,0,4500,0,16,...,1.52,120,500,0,120,0,M,A,A,0
1,35,9,9,0,0,1,0,3700,0,9,...,2.14,100,800,0,0,0,M,C,B,0
2,55,9,9,0,0,0,0,6400,0,1,...,0.50,50,700,0,0,0,M,C,B,0
3,55,42,0,42,0,5,0,8900,0,13,...,2.61,80,900,0,0,0,M,B,C,0
4,40,36,36,36,0,3,1,0,0,2,...,2.11,10,900,0,0,1,F,B,B,0


In [91]:
data.columns

Index(['AGE', 'TOT_LOAN', 'TOT_LOAN_CRD', 'LOAN_BNK', 'LOAN_CPT', 'CRDT_CNT',
       'GUARN_CNT', 'INCOME', 'LOAN_CRD_CNT', 'LATE_RATE', 'LATE_RATE_1Y',
       'INS_MON_MAX', 'CANCEL_CNT_1Y', 'CALL_TIME', 'TEL_COST_MON',
       'MOBILE_PRICE', 'SUSP_DAY', 'LATE_TEL', 'COMB_COMM', 'SEX',
       'PAY_METHOD', 'JOB', 'TARGET'],
      dtype='object')

In [92]:
#데이터 타입 확인 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43386 entries, 0 to 43385
Data columns (total 23 columns):
AGE              43386 non-null int64
TOT_LOAN         43386 non-null int64
TOT_LOAN_CRD     43386 non-null int64
LOAN_BNK         43386 non-null int64
LOAN_CPT         43386 non-null int64
CRDT_CNT         43386 non-null int64
GUARN_CNT        43386 non-null int64
INCOME           43386 non-null int64
LOAN_CRD_CNT     43386 non-null int64
LATE_RATE        43386 non-null int64
LATE_RATE_1Y     43386 non-null int64
INS_MON_MAX      43386 non-null float64
CANCEL_CNT_1Y    43386 non-null int64
CALL_TIME        43386 non-null float64
TEL_COST_MON     43386 non-null int64
MOBILE_PRICE     43386 non-null int64
SUSP_DAY         43386 non-null int64
LATE_TEL         43386 non-null int64
COMB_COMM        43386 non-null int64
SEX              43386 non-null object
PAY_METHOD       43386 non-null object
JOB              43386 non-null object
TARGET           43386 non-null int64
dtypes: fl

In [93]:
#object 타입 총 범위 확인 
print('Sex', data['SEX'].unique())
print('PAY_METHOD',data['PAY_METHOD'].unique())
print('JOB',data['JOB'].unique())

Sex ['M' 'F']
PAY_METHOD ['A' 'C' 'B' 'D']
JOB ['A' 'B' 'C' 'D']


In [94]:
#데이터 변경 dummy 말고 그냥 숫자 타입으로 변경 
data['SEX'] = data['SEX'].map({'M':0,'F':1})
data['PAY_METHOD'] = data['PAY_METHOD'].map({'A':0,'B':1,'C':2,'D':3})
data['JOB'] = data['JOB'].map({'A':0,'B':1,'C':2,'D':3})


In [95]:
data.head()

,AGE,TOT_LOAN,TOT_LOAN_CRD,LOAN_BNK,LOAN_CPT,CRDT_CNT,GUARN_CNT,INCOME,LOAN_CRD_CNT,LATE_RATE,...,CALL_TIME,TEL_COST_MON,MOBILE_PRICE,SUSP_DAY,LATE_TEL,COMB_COMM,SEX,PAY_METHOD,JOB,TARGET
0,40,6,6,6,0,0,0,4500,0,16,...,1.52,120,500,0,120,0,0,0,0,0
1,35,9,9,0,0,1,0,3700,0,9,...,2.14,100,800,0,0,0,0,2,1,0
2,55,9,9,0,0,0,0,6400,0,1,...,0.50,50,700,0,0,0,0,2,1,0
3,55,42,0,42,0,5,0,8900,0,13,...,2.61,80,900,0,0,0,0,1,2,0
4,40,36,36,36,0,3,1,0,0,2,...,2.11,10,900,0,0,1,1,1,1,0


In [96]:
############# 데이터 전처리   x , y  분리 
# split x variable to y variable
x = data.loc[:,data.columns != 'TARGET']
y = data.TARGET

In [97]:
# 정규화
X = preprocessing.minmax_scale(x)

In [98]:
#행열처리 하기 위함? 
X = pd.DataFrame(X)
X.columns = x.columns

In [99]:
#####데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30,random_state = 2017020558)

In [100]:
rf_clf = RandomForestClassifier(random_state=2017020558)

In [ ]:
np.random.seed(2017020558)
start = time.time()

param_dict = {'max_depth': [2,3,4,5],
             'max_features' : ['auto','sqrt','log2', None],
             'criterion': ['gini','entropy']}

cv_rf = GridSearchCV(rf_clf, cv=5,
                    param_grid=param_dict,
                    n_jobs = 3)

cv_rf.fit(X_train, y_train)

print('Best Parameters using grid search: \n',
     cv_rf.best_params_)
end = time.time()
print('Time taken in grid search: {:.2f}'.format(end-start))

In [ ]:
rf_clf.set_params(criterion='entropy', max_depth=4, max_features=None)

In [ ]:
rf_clf.set_params(oob_score=True)

min_estimators = 5
max_estimators = 200

error_rate = {}
for i in range(min_estimators, max_estimators + 1):
    rf_clf.set_params(n_estimators=i)
    rf_clf.fit(X_train, y_train)
    
    oob_error = 1 - rf_clf.oob_score_
    error_rate[i] = oob_error

In [ ]:
error_rate

In [ ]:
oob_series = pd.Series(error_rate)

In [ ]:
type(oob_series)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
oob_series.plot(kind='line',
                color = 'red')
plt.xlabel('n_estimators')
plt.ylabel('OOB Error Rate')
plt.title('OOB Error Rate Across various Forest sizes \n(From 5 to 200trees)')

In [ ]:
print('OOB Error rate for 100 trees is: {0:.5f}'.format(oob_series[100]))

In [ ]:
rf_clf.set_params(n_estimators=100,
                  bootstrap = True,
                  warm_start=False, 
                  oob_score=False)

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
imp = rf_clf.feature_importances_
imp = pd.DataFrame([X.columns, imp]).T
imp.columns = ['variables', 'importance']
sorted_idx = np.argsort(imp['importance'])[::-1]
imp = imp.iloc[sorted_idx,:]
print(imp[imp['importance'] > 0.01])

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.barplot(x='variables', y='importance', data=imp, ax=ax)
plt.setp(ax.get_xticklabels(), rotation=90)
plt.show(fig)

In [ ]:
y_test_pred = rf_clf.predict(X_test)
print('- Accuracy (Test) : {:.4}'.format(accuracy_score(y_test, y_test_pred)))
print('- Precision (Test) : {:.4}'.format(precision_score(y_test, y_test_pred)))
print('-Recall (Test) : {:.4}'.format(recall_score(y_test, y_test_pred)))
print('- F1 score (Test) : {:.4}'.format(f1_score(y_test, y_test_pred)))


In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
cm_test = pd.DataFrame(cm_test, columns=['1', '0'])
sns.heatmap(data=cm_test, annot=True, annot_kws={'size': 15})

In [ ]:
cm=confusion_matrix(y_test,y_test_pred)
cm

In [ ]:
#정확도
acc=(cm[0,0] + cm[1,1]) /sum(sum(cm[:,:]))
#민감도
sencifivity = cm[1,1]/sum(cm[1])
#특이도
specificity= cm[0,0]/sum(cm[0])
  

In [ ]:
print('-Recall (Test) : {:.4}'.format(sencifivity))
print('-specificity (Test) : {:.4}'.format(specificity))

In [ ]:
import math
print('-GM (Test) : {:.4}'.format(math.sqrt(sencifivity * specificity)))

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import OneSidedSelection

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

In [ ]:
def plot_(X_resampled, y_resampled, remove = True):
    # visualization 
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    
    if remove == True:
        idx_samples_removed = np.setdiff1d(np.arange(X_train.shape[0]),
                                       idx_resampled)
    
        idx_class_0 = y_resampled == 0
        plt.scatter(X_resampled[idx_class_0, 0], X_resampled[idx_class_0, 1],
                    alpha=.5, label='Class #0')
        plt.scatter(X_resampled[~idx_class_0, 0], X_resampled[~idx_class_0, 1],
                    alpha=.5, label='Class #1')
        plt.scatter(X_train[idx_samples_removed, 0], X_train[idx_samples_removed, 1],
                    alpha=.5, label='Removed samples')
    
    else:    
        
        idx_class_0 = y_resampled == 0
        plt.scatter(X_resampled[idx_class_0, 0], X_resampled[idx_class_0, 1],
                    alpha=.5, label='Class #0')
        plt.scatter(X_resampled[~idx_class_0, 0], X_resampled[~idx_class_0, 1],
                    alpha=.5, label='Class #1')

    # make nice plotting
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    #ax.set_xlim([-5, 5])
    #ax.set_ylim([-5, 5])
    
    #plt.title('Under-sampling using random under-sampling')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
###############################################################################
### SMOTE ('regular')

sm = SMOTE(k_neighbors = 1, kind = 'regular')
X_resampled, y_resampled = sm.fit_sample(X_train, y_train)

plot_(X_train, y_train, remove = False)
plot_(X_resampled, y_resampled, remove = True)


rf_clf.fit(X_resampled, y_resampled)

sm_rf_clf = rf_clf.fit(X_resampled, y_resampled)

y_resampled_pred = rf_clf.predict(X_resampled)
print('- Accuracy (Test) : {:.4}'.format(accuracy_score(y_resampled, y_resampled_pred)))
print('- Precision (Test) : {:.4}'.format(precision_score(y_resampled, y_resampled_pred)))
print('-Recall (Test) : {:.4}'.format(recall_score(y_resampled, y_resampled_pred)))
print('- F1 score (Test) : {:.4}'.format(f1_score(y_resampled, y_resampled_pred)))

sm_ = confusion_matrix(y_resampled, y_resampled_pred)
#정확도
acc=(sm_[0,0] + sm_[1,1]) /sum(sum(sm_[:,:]))
#민감도
sencifivity = sm_[1,1]/sum(sm_[1])
#특이도
specificity= sm_[0,0]/sum(sm_[0])
print('-Recall (Test) : {:.4}'.format(sencifivity))
print('-specificity (Test) : {:.4}'.format(specificity))
print('-GM (Test) : {:.4}'.format(math.sqrt(sencifivity * specificity)))